In [1]:
import tritonclient.grpc as grpcclient
import json
import dlib
from PIL import Image, ImageDraw
from pillow_heif import register_heif_opener
import numpy as np
from tqdm.auto import tqdm
from concurrent.futures import ThreadPoolExecutor, wait, FIRST_COMPLETED, as_completed

register_heif_opener()

In [ ]:
model_name = "dlib_face_descriptor"

In [ ]:
def submit_to_triton(image_data, image_wise_bboxes, request_id=""):
    inputs = []
    outputs = []
    image_wise_bboxes = np.array(
        [float(eval(i)) for i in image_wise_bboxes], dtype="float32"
    ).reshape((1, 4))

    inputs.append(grpcclient.InferInput("raw_image_array", image_data.shape, "UINT8"))
    inputs.append(grpcclient.InferInput("bboxes", [1, 4], "FP32"))

    inputs[0].set_data_from_numpy(image_data)
    inputs[1].set_data_from_numpy(image_wise_bboxes)

    outputs = [grpcclient.InferRequestedOutput("face_descriptor")]

    return triton_client.infer(
        model_name, inputs, outputs=outputs, request_id=request_id
    )


def parse_model_grpc(model_metadata, model_config):
    """
    Check the configuration of a model to make sure it meets the
    requirements for an image classification network (as expected by
    this client)
    """
    if len(model_metadata.inputs) != 2:
        raise Exception("expecting 2 input, got {}".format(len(model_metadata.inputs)))

    if len(model_config.input) != 2:
        raise Exception(
            "expecting 2 input in model configuration, got {}".format(
                len(model_config.input)
            )
        )

    input_metadata = model_metadata.inputs
    output_metadata = model_metadata.outputs

    return (input_metadata, output_metadata, model_config.max_batch_size)

In [ ]:
triton_client = grpcclient.InferenceServerClient(url=url, verbose=False)

model_metadata = triton_client.get_model_metadata(
    model_name=model_name, model_version=model_version
)

model_config = triton_client.get_model_config(
    model_name=model_name, model_version=model_version
).config

input_metadata, output_metadata, max_batch_size = parse_model_grpc(
    model_metadata, model_config
)

max_workers = max_batch_size * model_config.instance_group[0].count
input_names = [i.name for i in input_metadata]
output_names = [i.name for i in output_metadata]

In [ ]:
with open(facedetect_data_filename) as f:
    facedetect_data = json.load(f)

In [ ]:
pbar = tqdm(total=len(facedetect_data))
futures = []
ids = []
results = []
with ThreadPoolExecutor(max_workers=max_workers) as executor:
    for chunk in chunked(facedetect_data, 1000):
        futures = []
        for items in chunk:
            pbar.update()
            if items["faces"][0]["probability"] != "0":
                image = load_image(items["filename"])
                for idx, face in enumerate(items["faces"]):
                    request_id = "{}-{}".format(items["filename"], idx)
                    image_wise_bboxes = tuple(face["bbox"].values())
                    futures.append(
                        executor.submit(
                            submit_to_triton,
                            image,
                            image_wise_bboxes,
                            request_id=request_id,
                        )
                    )
        for future in as_completed(futures):
            result = future.result()
            ids.append(result.get_response().id)
            results.append(dlib.vector(result.as_numpy(output_names[0]).squeeze()))

In [ ]:
len(results)

In [ ]:
# results = []

# for idx, row in tqdm(
#     df.iterrows(),
#     desc="Generating Descriptors",
#     total=df.index.shape[0],
# ):
#     filename = row["filename"]
#     bboxes = np.expand_dims(np.array((row["bbox.x1"], row["bbox.y1"], row["bbox.x2"], row["bbox.y2"]), dtype="float32"), axis=0)
#     result = submit_to_triton(filename, bboxes, request_id=filename)
#     results.append(result.as_numpy(output_names[0]).reshape(-1, 128))

In [ ]:
labels = dlib.chinese_whispers_clustering(results, 0.5)

In [ ]:
import regex

In [ ]:
samp = "/data/photos/2013-grand-haven/grand-times-july-2013-little-nikon/102nikon/dscn1777.jpeg-1"

idx, _, filename = regex.split("(?r)(-+)", samp, maxsplit=1)

In [ ]:
for k, v, l in zip(ids, results, labels):
    idx, _, filename = regex.split("(?r)(-+)", k, maxsplit=1)

In [ ]:
for items in facedetect_data:
    if items["faces"][0]["probability"] == "0":
        pass
    else:
        for k, v, l in zip(ids, results, labels):
            idx, _, filename = regex.split("(?r)(-+)", k, maxsplit=1)